# ANOVA – decoding performance

This notebook runs a repeated-measures ANOVA on decoding accuracy with within-subject factors:
- stimulus type (Classic vs Gabor)
- contrast level

Before running:
- Set the `ROOT` path below to your local directory
- This notebook assumes that decoding performance has been generated (`decoding.ipynb`)

Note: `sub-004` is excluded due to channel issues / data quality (see thesis Methods).

In [1]:
import os
import pandas as pd
import pingouin as pg
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon

ROOT = r"C:\Users\donja\Desktop\Thesis"
RESULTS_DEC = os.path.join(ROOT, "Results", "decoding")

df = pd.read_csv(os.path.join(RESULTS_DEC, "decoding_metrics_all_subjects.csv"))
df_no4 = df[df["subject"] != "sub-004"].copy()

df_anova = (
    df_no4
    .groupby(["subject", "stim_type", "contrast"], as_index=False)["accuracy"]
    .mean()
)

print(df_anova.shape)
df_anova.head()

(48, 4)


,subject,stim_type,contrast,accuracy
0,sub-002,classic,10,0.842857
1,sub-002,classic,20,0.966667
2,sub-002,classic,30,1.000000
3,sub-002,classic,40,1.000000
4,sub-002,classic,60,1.000000


In [2]:
df_anova.groupby("stim_type")["accuracy"].agg(["mean", "std"]) * 100

,mean,std
stim_type,,
classic,91.607143,11.925706
grating,65.238095,39.591974


In [3]:
#10% Gabor was close to chance-level in figures, so mean and std were calculated
df_anova[(df_anova["stim_type"] == "grating") &(df_anova["contrast"] == 10)]["accuracy"].agg(["mean", "std"]) * 100

mean    5.238095
std     5.302633
Name: accuracy, dtype: float64

In [4]:
contrast_means = df_anova.groupby("contrast")["accuracy"].mean() * 100

contrast_means.loc[[contrast_means.index.min(), contrast_means.index.max()]]

print(contrast_means)

contrast
10     42.619048
20     76.309524
30     84.404762
40     83.333333
60     89.642857
100    94.226190
Name: accuracy, dtype: float64


In [5]:
df_anova["stim_type"] = df_anova["stim_type"].astype("category")
df_anova["contrast"] = df_anova["contrast"].astype("category")
df_anova["subject"] = df_anova["subject"].astype("category")

aov_accuracy = pg.rm_anova(
    data=df_anova,
    dv="accuracy",
    within=["stim_type", "contrast"],
    subject="subject",
    detailed=True
)

aov_accuracy

C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\pingouin\distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\pingouin\distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,stim_type,0.834392,1,3,0.834392,12.511536,0.038442,0.038442,0.304970,1.000000
1,contrast,1.377527,5,15,0.275505,17.821071,0.000008,0.012925,0.420090,0.259099
2,stim_type * contrast,0.653298,5,15,0.130660,6.385616,0.002282,0.071344,0.255705,0.236718


In [6]:
posthoc_stim = pg.pairwise_ttests(
    data=df_anova,
    dv="accuracy",
    within="stim_type",
    subject="subject",
    padjust="bonf"
)
posthoc_stim

C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\pingouin\pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,stim_type,classic,grating,True,True,3.537165,3.0,two-sided,0.038442,2.847,1.178516


## Post-hoc comparisons: Classic vs Gabor per contrast level

To further investigate the interaction between stimulus type and contrast, post-hoc paired comparisons were performed between Classic and Gabor stimuli
separately for each contrast level. Paired Wilcoxon tests were used due to the small sample size (N = 4). To control for multiple comparisons across contrast levels, Bonferroni correction was applied to the resulting p-values.

In [7]:
def posthoc_classic_vs_gabor_wilcoxon(df_wide):
    df_wide = df_wide.copy()

    results = []
    for contrast in sorted(df_wide["contrast"].unique()):
        df_c = df_wide[df_wide["contrast"] == contrast]

        # Pivot to ensure pairing by subject
        piv = df_c.pivot(index="subject", columns="stim_type", values="accuracy")

        if not {"classic", "grating"}.issubset(set(piv.columns)):
            raise ValueError(
                f"Missing classic/grating at contrast {contrast}. "
                f"Found: {list(piv.columns)}"
            )

        x = piv["classic"].astype(float).values
        y = piv["grating"].astype(float).values

        diffs = y - x
        diffs = diffs[~np.isnan(diffs)] 

        mean_diff = float(np.mean(diffs))
        median_diff = float(np.median(diffs))

        # Wilcoxon signed-rank test (paired)
        w_stat, p = wilcoxon(diffs, zero_method="wilcox", alternative="two-sided")

        results.append({
            "contrast": int(contrast),
            "N": int(len(diffs)),
            "mean_diff_gabor_minus_classic": mean_diff,
            "median_diff_gabor_minus_classic": median_diff,
            "W": float(w_stat),
            "p_uncorrected": float(p),
        })

    res_df = pd.DataFrame(results)

    # Multiple comparisons correction - Bonferroni
    m = len(res_df)
    res_df["p_bonferroni"] = np.minimum(res_df["p_uncorrected"] * m, 1.0)
    
    return res_df

posthoc_df = posthoc_classic_vs_gabor_wilcoxon(df_anova)
posthoc_df

C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\scipy\stats\_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\scipy\stats\_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
C:\Users\donja\miniconda3\envs\thesis_env\lib\site-packages\scipy\stats\_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


,contrast,N,mean_diff_gabor_minus_classic,median_diff_gabor_minus_classic,W,p_uncorrected,p_bonferroni
0,10,4,-0.747619,-0.800000,0.0,0.125000,0.750000
1,20,4,-0.314286,-0.285714,0.0,0.125000,0.750000
2,30,4,-0.202381,-0.064286,0.0,0.108809,0.652857
3,40,4,-0.164286,-0.030952,1.0,0.250000,1.000000
4,60,4,-0.133333,-0.045238,0.0,0.108809,0.652857
5,100,4,-0.020238,-0.016667,1.0,0.285049,1.000000
